# Imports

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
import glob
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns


2025-02-28 16:59:38.857963: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-28 16:59:38.866462: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-28 16:59:38.886323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740758378.919897   25397 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740758378.929188   25397 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-28 16:59:38.979793: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Configurations

In [2]:
IMG_HEIGHT = 256
IMG_WIDTH = 512
NUM_CLASSES = 9
BATCH_SIZE = 2


# Chargement des chemins

In [3]:
# Train
images_path_train = sorted(glob.glob("../data/raw/leftImg8bit/train/*/*.png"))
masks_path_train = sorted(glob.glob("../data/raw/gtFine/train/*/*_labelIds.png"))

# Validation
images_path_val = sorted(glob.glob("../data/raw/leftImg8bit/val/*/*.png"))
masks_path_val = sorted(glob.glob("../data/raw/gtFine/val/*/*_labelIds.png"))

# Test
images_path_test = sorted(glob.glob("../data/raw/leftImg8bit/test/*/*.png"))
masks_path_test = sorted(glob.glob("../data/raw/gtFine/test/*/*_labelIds.png"))


# Sampling n%

In [4]:
import random

# Pourcentage de données à conserver (par exemple, 10 %)
percentage_to_keep = 0.01

# Effectuer l'échantillonnage sur les données train
num_samples_to_keep = int(len(images_path_train) * percentage_to_keep)
sampled_indices = random.sample(range(len(images_path_train)), num_samples_to_keep)
sampled_images_path_train = [images_path_train[i] for i in sampled_indices]
sampled_masks_path_train = [masks_path_train[i] for i in sampled_indices]

# Remplacer les chemins d'origine par les chemins échantillonnés pour le train
images_path_train = sampled_images_path_train
masks_path_train = sampled_masks_path_train

# Vérifier le nombre d'images et de masques restants pour le train
print(f"Nombre d'images après échantillonnage (train) : {len(images_path_train)}")
print(f"Nombre de masques après échantillonnage (train) : {len(masks_path_train)}")

# Afficher les tailles des datasets
print(f"Taille du dataset d'entraînement : {len(images_path_train)}")
print(f"Taille du dataset de validation : {len(images_path_val)}")
print(f"Taille du dataset de test : {len(images_path_test)}")


Nombre d'images après échantillonnage (train) : 29
Nombre de masques après échantillonnage (train) : 29
Taille du dataset d'entraînement : 29
Taille du dataset de validation : 500
Taille du dataset de test : 1525


# Class Mapping

In [5]:
CLASS_MAPPING = {
    0: 0,    # 'unlabeled'            -> background
    1: 0,    # 'ego vehicle'          -> background
    2: 0,    # 'rectification border' -> background
    3: 0,    # 'out of roi'           -> background
    4: 0,    # 'static'               -> background
    5: 0,    # 'dynamic'              -> background
    6: 0,    # 'ground'               -> background
    7: 1,    # 'road'         -> flat
    8: 2,    # 'sidewalk'            -> flat
    9: 0,    # 'parking'              -> background
    10: 0,   # 'rail track'           -> background
    11: 3,   # 'building'             -> construction
    12: 0,   # 'wall'         -> construction
    13: 8,   # 'fence'                -> construction
    14: 0,   # 'guard rail'           -> background
    15: 0,   # 'bridge'               -> background
    16: 0,   # 'tunnel'               -> background
    17: 0,    # 'pole'          -> object
    18: 0,    # 'polegroup'            -> object
    19: 0,    # 'traffic light'        -> object
    20: 0,    # 'traffic sign'         -> object
    21: 4,   # 'vegetation'           -> nature
    22: 0,   # 'terrain'              -> nature
    23: 5,   # 'sky'                  -> sky
    24: 6,   # 'person'               -> human
    25: 6,   # 'rider'                -> human
    26: 7,   # 'car'                  -> vehicle
    27: 7,   # 'truck'                -> vehicle
    28: 7,   # 'bus'                  -> vehicle
    29: 7,   # 'caravan'              -> vehicle
    30: 7,   # 'trailer'              -> vehicle
    31: 7,   # 'train'                -> vehicle
    32: 7,   # 'motorcycle'           -> vehicle
    33: 7,   # 'bicycle'              -> vehicle
    -1: 0     # 'license plate'        -> background
}


# Fonctions utilitaires

In [6]:
def remap_classes(mask, mapping):
    remapped_mask = np.zeros_like(mask, dtype=np.uint8)
    for old_class, new_class in mapping.items():
        remapped_mask[mask == old_class] = new_class
    return remapped_mask

def get_city_name(image_path):
    return image_path.split('/')[-2]

def get_image_name(image_path):
    return image_path.split('/')[-1].split('_leftImg8bit')[0]

def load_and_preprocess_image(image_path,
                              mask_path,
                              class_mapping,
                              img_height,
                              img_width):
    
    # Chargement de l'image
    image = cv2.imread(image_path)
    # Conversion en couleur
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Chargement du masque
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

    # Resize
    image = cv2.resize(image, (img_width, img_height))
    mask = cv2.resize(mask, (img_width, img_height), interpolation=cv2.INTER_NEAREST)

    # Appliquer le remappage des classes
    remapped_mask = remap_classes(mask, class_mapping)

    # Normalize image
    image = image.astype("float32") / 255.0

    return image, remapped_mask


# Data Generator

In [7]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self,
                 images_path,
                 masks_path,
                 batch_size,
                 img_height,
                 img_width,
                 num_classes,
                **kwargs):

        super().__init__(**kwargs)
        self.images_path = images_path
        self.masks_path = masks_path
        self.batch_size = batch_size
        self.img_height = img_height
        self.img_width = img_width
        self.num_classes = num_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.images_path) / self.batch_size))

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = (idx + 1) * self.batch_size

        batch_images_path = self.images_path[start:end]
        batch_masks_path = self.masks_path[start:end]

        batch_images = []
        batch_masks = []

        for i in range(len(batch_images_path)):
            image, mask = load_and_preprocess_image(batch_images_path[i],
                                                    batch_masks_path[i],
                                                    CLASS_MAPPING,
                                                    self.img_height,
                                                    self.img_width)
            batch_images.append(image)
            batch_masks.append(mask)

        batch_images = np.array(batch_images)
        batch_masks = np.array(batch_masks)

        batch_masks = np.expand_dims(batch_masks, -1)
        batch_masks = tf.keras.utils.to_categorical(batch_masks,
                                                    num_classes=self.num_classes)
        
        return batch_images, batch_masks

    def on_epoch_end(self):
        # Optionally shuffle the data at the end of each epoch
        pass


# Modèle UNet

In [8]:
def build_unet(img_height, img_width, num_classes):
    inputs = keras.layers.Input(shape=(img_height, img_width, 3))

    # Encoder
    conv1 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = keras.layers.MaxPooling2D((2, 2))(conv1)

    conv2 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = keras.layers.MaxPooling2D((2, 2))(conv2)

    # Bottleneck
    conv3 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)

    # Decoder
    up4 = keras.layers.UpSampling2D((2, 2))(conv3)
    merge4 = keras.layers.concatenate([conv2, up4], axis=-1)
    conv4 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(merge4)

    up5 = keras.layers.UpSampling2D((2, 2))(conv4)
    merge5 = keras.layers.concatenate([conv1, up5], axis=-1)
    conv5 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(merge5)

    # Output
    outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv5)

    model = keras.models.Model(inputs=inputs, outputs=outputs)
    return model


# Entraînement du modèle

In [9]:
# Build model
model = build_unet(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators
train_generator = DataGenerator(images_path_train,
                                masks_path_train,
                                BATCH_SIZE,
                                IMG_HEIGHT,
                                IMG_WIDTH,
                                NUM_CLASSES)

val_generator = DataGenerator(images_path_val,
                              masks_path_val,
                              BATCH_SIZE,
                              IMG_HEIGHT,
                              IMG_WIDTH,
                              NUM_CLASSES)

# Define callbacks
checkpoint = ModelCheckpoint("../models/unet_model.h5",
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='max')

early_stopping = EarlyStopping(monitor='val_accuracy',
                               patience=10,
                               verbose=1,
                               restore_best_weights=True)

# Train model
model.fit(train_generator,
          validation_data=val_generator,
          epochs=10,
          callbacks=[checkpoint, early_stopping])

print("Model Building and Training Complete!")

2025-02-28 16:59:43.090511: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/10


2025-02-28 16:59:49.140546: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 905969664 exceeds 10% of free system memory.
2025-02-28 16:59:49.140631: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 905969664 exceeds 10% of free system memory.
2025-02-28 16:59:50.653179: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 452984832 exceeds 10% of free system memory.
2025-02-28 16:59:50.653304: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 452984832 exceeds 10% of free system memory.


 1/14 ━━━━━━━━━━━━━━━━━━━━ 2:18 11s/step - accuracy: 0.0000e+00 - loss: 2.2164

2025-02-28 16:59:56.638548: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 905969664 exceeds 10% of free system memory.


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.2695 - loss: 2.0813       
Epoch 1: val_accuracy improved from -inf to 0.32918, saving model to unet_model.h5


14/14 ━━━━━━━━━━━━━━━━━━━━ 509s 38s/step - accuracy: 0.2728 - loss: 2.0770 - val_accuracy: 0.3292 - val_loss: 1.9214
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.3288 - loss: 1.9148  

KeyboardInterrupt: 

# Évaluation du modèle

In [ ]:
# Evaluate the model on the test set
print("Génération des prédictions sur le jeu de test :")
num_test_samples = len(images_path_test)
test_generator = DataGenerator(images_path_test,
                                masks_path_test,
                                BATCH_SIZE,
                                IMG_HEIGHT,
                                IMG_WIDTH,
                                NUM_CLASSES)

predictions = model.predict(test_generator, steps=num_test_samples // BATCH_SIZE, verbose=1)
predicted_labels = np.argmax(predictions, axis=-1)

# Get true labels from the test set
true_labels = []
for i in range(len(test_generator)):
    _, batch_masks = test_generator[i]
    # Convert categorical labels to integers
    batch_true_labels = np.argmax(batch_masks, axis=-1)
    true_labels.extend(batch_true_labels.flatten())
true_labels = np.array(true_labels)

# Keep only the number of samples that are equal to the predictions
true_labels = true_labels[:predicted_labels.size]

# Flatten predicted_labels array
predicted_labels = predicted_labels.flatten()

# Create confusion matrix
print("Création de la matrice de confusion :")
confusion_mtx = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
print("Affichage de la matrice de confusion :")
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

print("Model Evaluation Complete!")